# Itérateurs, générateurs, décorateurs
### programmation fonctionnelle et Python

Je ne vais pas vous faire un cours sur la programmation fonctionnelle. Je vous invite cependant à vous intéresser à ce paradigme de programmation ou à jetter un œil au vénérable [Lisp](https://fr.wikipedia.org/wiki/Lisp), à [Haskell](https://www.haskell.org/) ou [OCaml](https://ocaml.org/).

En Python tout est objet, ça vous le savez. Vous savez aussi que Python est un langage multi-paradigme. Vous pouvez programmer dans un style procédural, en objet ou dans un style fonctionnel. Qu'est-ce que cela signifie un style fonctionnel ?  
Vous ne pourrez pas produire de programmation fonctionnelle « pure » mais vous pouvez vous en approcher en privilégiant les fonctions sans effet de bord (pas de changement d'état, par exemple dans les structures de données), en évitant les variables globales, ou encore en utilisant des fonctions d'ordre supérieur (c-a-d des fonctions qui acceptent des fonctions comme arguments ou qui renvoient des fonctions).  
Vous pouvez aussi apprendre à vous servir des itérateurs, des générateurs (ce qu'on fera ici) puis, si vous voulez, aller plus loin avec les modules [itertools](https://docs.python.org/3/library/itertools.html#module-itertools) et [functools](https://docs.python.org/3/library/functools.html#module-functools)

# Les itérateurs

Les itérateurs vous connaissez, vous en utilisez tous les jours avec les itérables que sont les chaînes, les listes ou les dictionnaires.

In [ ]:
numbers = [1, 2, 3, 4, 5]
for it in numbers:
    print(it)

Mais alors on peut utiliser des itérateurs avec tous les objets ? Non.  
Les itérateurs sont des objets qui représentent un flux de données. Pour être itérable un objet doit implémenter la fonction `__next()__`. Cette fonction peut aussi s'appeler avec `next()`, elle ne reçoit pas d'argument, renvoie le prochain élément, et si plus d'élément renvoie l'exception `StopIteration`.

Pour savoir si un objet peut être un itérateur vous lui appliquez la fonction `iter()`, si elle ne renvoie pas d'exception c'est bon.

In [19]:
nb = 42
nb_iter = iter(nb) # c'est pas bon

TypeError: 'int' object is not iterable

In [30]:
numbers = [1, 2, 3, 4, 5]
nb_iter = iter(numbers) # c'est bon
# on peut utiliser __next__()
nb_iter.__next__()

1

In [31]:
next(nb_iter) # autre façon

2

Un itérateur est un flux, vous pouvez accéder aux éléments les uns après les autres mais pas revenir en arrière ou faire une copie. Si vous voulez accèder à nouveau au flux vous devez utiliser un nouvel itérateur. C'est le cas pour la lecture d'un fichier par exemple : vous ne pouvez pas lire à nouveau l'objet fichier si vous l'avez déjà fait.

# Les générateurs

Les générateurs sont très simples à utiliser et très puissants. Ils vous permettront d'optimiser votre code à moindre frais. Alors pourquoi se priver ?

Imaginons que je veuille extraire d'une liste de mots la liste des mots comportants le caractère 'a'. Je vais écrire une fonction.

In [3]:
def with_a(words):
    """
    Reçoit une liste de mots et renvoie la liste des mots contenant le car. 'a'
    """
    res = []
    for word in words:
        if 'a' in word:
            res.append(word)
    return res
    

In [5]:
mots = ["le", "petit", "chat", "est", "mort", "ce", "matin"]
mots_a = with_a(mots)
print("\n".join(mots_a))

chat
matin


Jusque là rien de méchant. Comme il est question d'optimisation je vais mesurer le temps de traitement avec `timeit`.  
ipython est plein de magie, `%time` hup hup hup barbatruc et voilà.

In [6]:
%time mots_a = with_a(mots)
mots_big = mots * 1000000
%time mots_a = with_a(mots_big)

CPU times: user 24 µs, sys: 0 ns, total: 24 µs
Wall time: 37 µs
CPU times: user 229 ms, sys: 150 µs, total: 230 ms
Wall time: 229 ms


Comme on pouvait s'y attendre le temps d'exécution de la fonction augmente avec la taille de la liste initiale.  
Voyons ce que ça donne avec un générateur. Construire un générateur c'est simple : vous remplacez `return` par `yield` dans votre fonction.  
C'est tout ? C'est tout.  

<small>Vous pouvez quand même en apprendre plus en lisant la [PEP 255](https://www.python.org/dev/peps/pep-0255/) si vous aimez ça.</small>

In [37]:
def gen_with_a(words):
    """
    Reçoit une liste de mots et renvoie les mots contenant le car. 'a' sous forme de générateur
    """
    for word in words:
        if 'a' in word:
            yield(word)

In [8]:
mots_big = mots * 100
%time mots_a = with_a(mots_big)
%time mots_a_gen = gen_with_a(mots_big)

CPU times: user 3.9 ms, sys: 0 ns, total: 3.9 ms
Wall time: 3.91 ms
CPU times: user 11 µs, sys: 0 ns, total: 11 µs
Wall time: 17.9 µs


😲 !!!!!!!!!  
Oui c'est de la magie. Enfin c'est plutôt de la triche, regardez :

In [9]:
print(f"mots_a is a {type(mots_a)}")
print(f"mots_a_gen is a {type(mots_a_gen)}")
import sys
print(f"Taille de mots_a : {sys.getsizeof(mots_a)}")
print(f"Taille de mots_a_gen : {sys.getsizeof(mots_a_gen)}")

mots_a is a <class 'list'>
mots_a_gen is a <class 'generator'>
Taille de mots_a : 1680
Taille de mots_a_gen : 128


`mots_a_gen` n'est pas une liste, c'est un objet `generator`.  
Il ne stocke rien ou presque en mémoire, on ne peut pas connaître sa taille (essayez `len(mots_a_gen)` pour voir.  
Mais c'est un itérable, on peut le parcourir comme une liste. Par contre on ne peut pas les "trancher", on ne peut accéder à un élément d'index `i` comme pour une liste.  
Encore une différence d'avec les listes : vous ne pouvez parcourir un générateur qu'une seule fois.

Mais ça ça rappelle les itérateurs. Oui. Les générateurs permettent de créer des itérateurs sans se fatiguer. Une fonction classique reçoit des paramètres, calcule un truc avec et renvoie le résultat. Un générateur renvoie un itérateur qui donne accès à un flux de données.  
Comme tout itérateur vous pouvez le convertir en liste ou en tuple si vous voulez.

In [10]:
%time mots_a_gen = list(gen_with_a(mots_big))

CPU times: user 166 µs, sys: 25 µs, total: 191 µs
Wall time: 204 µs


Mais même sans tricher les générateurs demeurent très efficaces. Vous aurez compris qu'il vous est désormais chaudement recommandé de les utiliser. 

Si vous voulez en savoir plus sur la cuisine du truc vous pouvez utiliser le module `inspect`. Je vous conseille d'en lire la doc d'ailleurs : [https://docs.python.org/3/library/inspect.html](https://docs.python.org/3/library/inspect.html)

In [50]:
import inspect

mots_a_gen = gen_with_a(mots)
print(mots_a_gen)
print(inspect.getgeneratorstate(mots_a_gen))
print(next(mots_a_gen))
print(inspect.getgeneratorstate(mots_a_gen))
print(next(mots_a_gen))
print(inspect.getgeneratorstate(mots_a_gen))
print(next(mots_a_gen))

<generator object gen_with_a at 0x7f2b9997a050>
GEN_CREATED
chat
GEN_SUSPENDED
matin
GEN_SUSPENDED


StopIteration: 

In [51]:
inspect.getgeneratorstate(mots_a_gen)

'GEN_CLOSED'

Vous pouvez aussi utiliser des générateurs en compréhension, à la manière des listes en compréhension : 

In [11]:
[mot for mot in mots if 'a' in mot]

['chat', 'matin']

In [12]:
(mot for mot in mots if 'a' in mot)

<generator object <genexpr> at 0x7f2ba85ebd50>

# Encore un peu de fonctionnel : fonctions lambda, `map` et `filter`

`map`et `filter` sont typiquement des fonctions qui viennent des langages fonctionnels. Elles renvoien toutes les deux des itérateurs.

  - `map` permet d'appliquer un traitement sur chaque élément d'un itérable
  - `filter` filtre les éléments d'un itérable en fonction d'une condition

Oui on peut faire tout ça avec les listes en compréhension. C'est même plus pythonique, vous allez donc continuer à utiliser les listes en compréhension plutôt que `map` et `filter`.

In [67]:
def carre(x):
    return x**2

numbers = [1, 2, 3, 4, 5]
for it in map(carre, numbers):
    print(it)
# ou aussi
list(map(carre, numbers))

1
4
9
16
25


[1, 4, 9, 16, 25]

In [69]:
[it**2 for it in numbers] # so pythonic

[1, 4, 9, 16, 25]

In [74]:
def is_even(x):
    return not(x%2)

for it in filter(is_even, numbers):
    print(it)
# ou aussi
list(filter(is_even, numbers))

2
4


[2, 4]

In [75]:
[not(it % 2) for it in numbers]

[False, True, False, True, False]

C'est un peu fastidieux d'écrire ces petites fonctions pour utiliser `map` et `filter`. Avec les fonctions lambda, Python offre un moyen d'écrire des petites fonctions, de leur passer des paramètres et d'en faire des fonctions anonymes. Oui des fonctions anonymes, elles n'ont pas de nom quoi. Encore un truc qui vient de la programmation fonctionnelle, on en utilise plein en Javascript par exemple.

In [90]:
for it in map(lambda x: x**2, numbers):
    print(it)

1
4
9
16
25


Ici on a bien une fonction qui est paramètre d'une autre fonction (`map`). On utilise souvent des fonctions lambda avec `sorted`, typiquement pour trier un dictionnaire par valeur comme vous le savez.

In [94]:
letters = {'a': 5, 'b': 2, 'c': 7, 'd':1, 'e':12}
for it, val in sorted(letters.items(), key=lambda item: item[1]):
    print(it, val)

d 1
b 2
a 5
c 7
e 12


# Les décorateurs

Les décorateurs ont été introduit avec la [PEP 318](https://www.python.org/dev/peps/pep-0318/) en 2003 dans la version 2.4 de Python.

Une fonction est un objet. Vous savez : en Python tout est objet. On peut passer une fonction en paramètre d'une fonction. Une fonction peut renvoyer une fonction en valeur de retour.

In [99]:
def salut():
    print("salut")

bonjour = salut # passage de la référence de l'objet (remember le cours sur les classes et les objets)
bonjour()

salut


Avec un décorateur on va emballer une fonction pour ajouter des fonctionnalités. Un décorateur reçoit en paramètre une fonction et l'emballe dans une autre.

In [103]:
def deco(func):
    def wrapper():
        print("salut", end=" ")
        func()
    return wrapper

def name():
    print("jean-michel")

obj = deco(name)
obj()

salut jean-michel


La PEP 318 a introduit le symbole '@'. Ça permet d'avoir une syntaxe plus simple, du code plus propre.

In [108]:
@deco
def name():
    print("jean-michel")
    
obj = name
obj()

salut jean-michel


Ce décorateur ne sert à rien, on est d'accord. Voici un exemple plus parlant avec un décorateur pour mesurer le temps d'exécution d'une fonction :

In [109]:
import time

def timer(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        value = func(*args, **kwargs)
        end = time.perf_counter()
        run_time = end - start
        print(f"Finished {func.__name__} in {run_time} secs"
        return value

    return wrapper

In [113]:
@timer
def doubled_and_add(num):
    res = sum([i*2 for i in range(num)])
    print(f"Result : {res}")

doubled_and_add(100000)

Result : 9999900000
Finished 'doubled_and_add' in 0.015 secs


In [111]:
doubled_and_add(1000000)

Result : 999999000000
Finished 'doubled_and_add' in 0.101 secs


Je ne suis pas persuadé que vous ayiez à écrire des décorateurs dans un avenir proche. Par contre vous serez certainement amenées à en utiliser. Si vous faîtes du web en Python, que ce soit avec [Django](https://www.djangoproject.com/) ou [Flask](https://flask.palletsprojects.com/en/1.1.x/), c'est certain.

Et tout de suite là maintenant vous allez en utiliser pour coder votre bot Discord 🎉🤖🥳. Nous allons utiliser le module [discord.py](https://discordpy.readthedocs.io/en/latest/index.html) et nous inspirer du tutoriel [https://realpython.com/how-to-make-a-discord-bot-python/](https://realpython.com/how-to-make-a-discord-bot-python/)